In [1]:
# Import libraries 
import extract_eye_features
import dill
import scipy
import os

# Import the path to Pi_util
import sys 
sys.path.append("/Users/zacharykelly/Documents/MATLAB/projects/lightLogger/raspberry_pi_firmware/utility")
import Pi_util
import natsort
import importlib

Loading DLC 3.0.0rc8...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [ ]:
# Generate a playable video
path_to_recording_chunks: str = "/Volumes/T7 Shield/FLIC_2004_gazeCalibration_tf_session1"
output_dir: str = "./STORAGE_FOLDER/VIDEO_FOLDER"
Pi_util.generate_playable_videos(path_to_recording_chunks, output_dir, apply_digital_gain=True, fill_missing_frames=True, debayer_images=True, pupil_image_rotation_correction=True)

In [ ]:
# Attempt to extract pupil + eyelid features from this video 
importlib.reload(extract_eye_features)
path_to_video: str = "./FLIC_200X_gazeCalibration_session1/P.avi"
eye_features, perimeter_dict = extract_eye_features.extract_eye_features(path_to_video, 
                                                                         is_grayscale=True, 
                                                                         visualize_results=True, 
                                                                         method='pylids',
                                                                         safe_execution=True,
                                                                         keypoint_threshold=-1
                                                                        )

In [ ]:
# Attempt to extract only pupil features from this video 
importlib.reload(extract_eye_features)
path_to_video: str = "/Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2005/walkIndoor/temporalFrequency/P.avi"
pupil_features, perimeter_dict = extract_eye_features.extract_pupil_features(path_to_video, 
                                                                             is_grayscale=True, 
                                                                             visualize_results=True, 
                                                                             method='pylids',
                                                                             safe_execution=True,
                                                                             keypoint_threshold=-1,
                                                                             visualization_output_filepath="/Users/zacharykelly/Desktop/FLIC_2005_walkIndoor_tf_visualizedPupilFeatures.avi"
                                                                            )

In [ ]:
# Define the input and output directories 
input_directory: str = "/Volumes/GKA spare/scriptedIndoorOutdoorVideos"
output_directory: str = "/Users/zacharykelly/Aguirre-Brainard Lab Dropbox/Zachary Kelly/FLIC_analysis/lightLogger/scriptedIndoorOutdoor"

# Iterate through the subjects in the input directory 
for subject_ID in natsort.natsorted(os.listdir(input_directory)):
    # Skip hidden files 
    if(subject_ID.startswith(".")):
        continue 

    if(subject_ID not in ("FLIC_2002", "FLIC_2005")):
        continue

    # Define the path to the subject 
    subject_ID_path: str = os.path.join(input_directory, subject_ID)

    # Iterate over the activites for this subject 
    for activity in natsort.natsorted(os.listdir(subject_ID_path)):
        # Skip hidden files 
        if(activity.startswith(".")):
            continue 

        # Skip activites we do not want 
        if(activity not in ("lunch", )):
            continue

        # Otherwise, construct the path to this activity 
        activity_path: str = os.path.join(subject_ID_path, activity)

        # Iterate over the modes of this activity 
        for mode in natsort.natsorted(os.listdir(activity_path)):
            # Skip hidden files 
            if(mode.startswith(".")):
                continue 
            
            # Skip spatial for now 
            if("spatial" in mode):
                continue 

            # Construct the path to the mode 
            mode_path: str = os.path.join(activity_path, mode)

            # Construct the path to the contrast video
            contrast_videos: list[str] = [filename
                                          for filename in os.listdir(mode_path)
                                          if "contrast" in filename
                                         ]
            assert len(contrast_videos) == 1, f"Subject: {subject_ID} | Activity: {activity} | {contrast_videos}"

            contrast_video: str = contrast_videos[0]
            additional_information: str = contrast_video.split("_")[-2]


            # Construct the path to the contrast video 
            contrast_video_path: str = os.path.join(mode_path, contrast_video)

            # Retrieve the perimeter dict
            pupil_features, perimeter_dict = extract_eye_features.extract_pupil_features(contrast_video_path, 
                                                                             is_grayscale=True, 
                                                                             visualize_results=False, 
                                                                             method='pylids',
                                                                             safe_execution=True,
                                                                             keypoint_threshold=-1,
                                                                             visualization_output_filepath=""
                                                                            )                                   
            # Define the perimeter output path 
            perimeter_output_path: str = os.path.join(output_directory, subject_ID, activity, mode, f"{subject_ID}_{activity}_tf_perimeter_{additional_information}.mat")

            print(f"Contrast video path: {contrast_video_path}")
            print(f"Perimeter output path: {perimeter_output_path}")

            # save the output 
            scipy.io.savemat(perimeter_output_path, {"perimeter": perimeter_dict})

Contrast video path: /Volumes/GKA spare/scriptedIndoorOutdoorVideos/FLIC_2002/lunch/temporalFrequency/FLIC_2002_lunch_contrast1x25gamma1_P.avi
Perimeter output path: /Users/zacharykelly/Aguirre-Brainard Lab Dropbox/Zachary Kelly/FLIC_analysis/lightLogger/scriptedIndoorOutdoor/FLIC_2002/lunch/temporalFrequency/FLIC_2002_lunch_tf_perimeter_contrast1x25gamma1.mat
Contrast video path: /Volumes/GKA spare/scriptedIndoorOutdoorVideos/FLIC_2005/lunch/temporalFrequency/FLIC_2005_lunch_contrast1x25gamma1_P.avi
Perimeter output path: /Users/zacharykelly/Aguirre-Brainard Lab Dropbox/Zachary Kelly/FLIC_analysis/lightLogger/scriptedIndoorOutdoor/FLIC_2005/lunch/temporalFrequency/FLIC_2005_lunch_tf_perimeter_contrast1x25gamma1.mat


In [ ]:
# Attempt to extract only eyelid features from this video 
eyelid_features: list[dict] = extract_eye_features.extract_pupil_features(path_to_video, 
                                                                          is_grayscale=True, 
                                                                          visualize_results=True, 
                                                                          method='pylids',
                                                                          safe_execution=True,
                                                                          keypoint_threshold=-1
                                                                         )

In [ ]:
scipy.io.savemat("/Users/zacharykelly/Desktop/FLIC_2005_walkIndoor_tf_perimeter.mat", {"perimeter": perimeter_dict})